In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-3
eval_iters = 100
eval_interval = 500
n_embd = 384
n_layer = 4
n_head =4
dropout = 0.2
device

'cuda'

In [3]:
with open('Wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [4]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

def encode(s): return [string_to_int[c] for c in s]

def decode(l): return ''.join([int_to_string[i] for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [28]:
class Feedforward(nn.Module):
    #simple linear layer followed by a non-linearity

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout), # drops a certain % of our neurons and makes them 0, needed to avoid overFitting
        )

    def forward(self, x):
        return self.net(x)
    
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape # batch, time, channel
        k = self.key(x)
        q = self.query(x)
        wei = q@k.transpose(-2, -1)*k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        #what is the mask doing, basically a TxT matrix
        # [1, -inf, -inf]
        # [1, 0.4, -inf]
        # [1, 0.4, 0.6] basically sequential guessing
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v=self.value(x)
        out=wei@v
        return out
    
class MultiHeadAttention(nn.Module):
    # multiple heads of self-attention in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) #heads in parallel
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, Features) -> (B, T, h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3) 4 features and 3 heads 
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    #transformer Decoder block
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) #self attention
        self.ffwd = Feedforward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

    
class GPT_LM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)]) ##Number of decoder blocks (sequential, not parallel)
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.ln_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T= index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.ln_head(x) # linear_transformation

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            index = torch.cat((index, index_next), dim=1)  # (B, T+1)
        return index
    
model = GPT_LM(vocab_size)
m = model.to(device)

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):

    if iter % 100 == 0:
        losses = estimate_loss()
        print(f'step: {iter}, loss {losses}')

    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step: 0, loss {'train': tensor(3.1448), 'val': tensor(3.1511)}
step: 100, loss {'train': tensor(3.1403), 'val': tensor(3.1532)}
step: 200, loss {'train': tensor(3.1370), 'val': tensor(3.1467)}
step: 300, loss {'train': tensor(3.1416), 'val': tensor(3.1539)}
step: 400, loss {'train': tensor(3.1381), 'val': tensor(3.1467)}
step: 500, loss {'train': tensor(3.1324), 'val': tensor(3.1499)}
step: 600, loss {'train': tensor(3.1365), 'val': tensor(3.1529)}
step: 700, loss {'train': tensor(3.1344), 'val': tensor(3.1540)}
step: 800, loss {'train': tensor(3.1352), 'val': tensor(3.1464)}
step: 900, loss {'train': tensor(3.1358), 'val': tensor(3.1527)}
step: 1000, loss {'train': tensor(3.1399), 'val': tensor(3.1512)}
step: 1100, loss {'train': tensor(3.1376), 'val': tensor(3.1542)}
step: 1200, loss {'train': tensor(3.1394), 'val': tensor(3.1484)}
step: 1300, loss {'train': tensor(3.1388), 'val': tensor(3.1505)}
step: 1400, loss {'train': tensor(3.1385), 'val': tensor(3.1497)}
step: 1500, loss {'tra

In [32]:
context = torch.zeros((1, 1), dtype = torch.long, device = device)
generateChars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generateChars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2